#### Fast Segment Anything Model (FastSAM)
- Mask Coeff:
- ProtoNet: 
- instance segmentation 분기가 장착된 object detector인 YOLOv8-seg를 기반
- SAM의 SA-1B 데이터셋을 활용
![img.jpg](https://user-images.githubusercontent.com/26833433/248551984-d98f0f6d-7535-45d0-b380-2e1440b52ad7.jpg)

추가 설명
- All-instance Segmentation과 Prompt-guided Selection의 두 단계로 구성
- Detection 분기는 카테고리와 boundary box를 출력하는 반면 
- Instance segmentation 분기는 k개의 마스크 계수와 함께 k개의 프로토타입 (FastSAM에서 32로 기본 설정됨)을 출력
  - `Instance segmentation 결과는 마스크 계수를 프로토타입과 곱한 다음 합산하여 얻는다`

#### FastSAM\Inference.py
- FastSAM\fastsam\prompt.py 의 class FastSAMPrompt 를 참고한다.

In [ ]:
import argparse
from fastsam import FastSAM, FastSAMPrompt 
import ast
import torch
from PIL import Image
from utils.tools import convert_box_xywh_to_xyxy

def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--model_path", type=str, default="./weights/FastSAM.pt", help="model")
    parser.add_argument("--img_path", type=str, default="./images/dogs.jpg", help="path to image file")
    parser.add_argument("--imgsz", type=int, default=1024, help="image size")
    parser.add_argument(
        "--iou", type=float, default=0.9, help="iou threshold for filtering the annotations",
    )
    parser.add_argument("--text_prompt", type=str, default=None, help='use text prompt eg: "a dog"' )
    parser.add_argument("--conf", type=float, default=0.4, help="object confidence threshold" )
    parser.add_argument("--output", type=str, default="./output/", help="image save path")
    parser.add_argument("--randomcolor", type=bool, default=True, help="mask random color")
    parser.add_argument("--point_prompt", type=str, default="[[0,0]]", help="[[x1,y1],[x2,y2]]")
    parser.add_argument("--point_label", type=str, default="[0]", help="[1,0] 0:background, 1:foreground",)

    parser.add_argument("--box_prompt", type=str, default="[[0,0,0,0]]", help="[[x,y,w,h],[x2,y2,w2,h2]] support multiple boxes")
    parser.add_argument(
        "--better_quality",
        type=str,
        default=False,
        help="better quality using morphologyEx",
    )
    device = torch.device("cuda"
        if torch.cuda.is_available() else "mps"
        if torch.backends.mps.is_available() else "cpu"
    )
    parser.add_argument(
        "--device", type=str, default=device, help="cuda:[0,1,2,3,4] or cpu"
    )
    parser.add_argument(
        "--retina",
        type=bool,
        default=True,
        help="draw high-resolution segmentation masks",
    )
    parser.add_argument(
        "--withContours", type=bool, default=False, help="draw the edges of the masks"
    )
    return parser.parse_args()

def main(args):
    model = FastSAM(args.model_path) # weight 경로를 입력하면, FastSAM(YOLO) 부모 클래스인 YOLO가 초기화 된다.
    args.point_prompt = ast.literal_eval(args.point_prompt)
    args.box_prompt = convert_box_xywh_to_xyxy(ast.literal_eval(args.box_prompt))
    args.point_label = ast.literal_eval(args.point_label)
    input = Image.open(args.img_path)
    input = input.convert("RGB")

    everything_results = model( # FastSAM.__call__(self, source=None, stream=False, **kwargs)
        input,                  # 즉, FastSAM.predict(source, stream, **kwargs) 호출. 그다음은??
        device=args.device,
        retina_masks=args.retina,
        imgsz=args.imgsz,
        conf=args.conf,
        iou=args.iou 
    )

    bboxes = None
    points = None
    point_label = None

    if args.box_prompt[0][2] != 0 and args.box_prompt[0][3] != 0:
            ann = prompt_process.box_prompt(bboxes=args.box_prompt)
            bboxes = args.box_prompt

    elif args.text_prompt != None:
        ann = prompt_process.text_prompt(text=args.text_prompt)

    elif args.point_prompt[0] != [0, 0]:
        ann = prompt_process.point_prompt( # 포인트 프롬프트의 경우 라벨이 필요하다.
            points=args.point_prompt, pointlabel=args.point_label
        )


if __name__ == "__main__":
    args = parse_args()
    main(args)

#### FastSAM\fastsam\model.py

In [ ]:
"""
FastSAM model interface.

Usage - Predict:
    from ultralytics import FastSAM

    model = FastSAM('last.pt')
    results = model.predict('ultralytics/assets/bus.jpg')
"""

from ultralytics.yolo.cfg import get_cfg
from ultralytics.yolo.engine.exporter import Exporter
from ultralytics.yolo.engine.model import YOLO
from ultralytics.yolo.utils import DEFAULT_CFG, LOGGER, ROOT, is_git_dir
from ultralytics.yolo.utils.checks import check_imgsz

from ultralytics.yolo.utils.torch_utils import model_info, smart_inference_mode
from .predict import FastSAMPredictor

class FastSAM(YOLO):

    @smart_inference_mode()
    def predict(self, source=None, stream=False, **kwargs): # ★★★ 가장 중요
        """
        YOLO 모델로 추론 수행

        Args:
            source (str | int | PIL | np.ndarray): 모든 형태의 입력 수용
            stream (bool): 입력의 stram 여부

        Returns:
            (List[ultralytics.yolo.engine.results.Results])
        """
        if source is None:
            source = ROOT / 'assets' if is_git_dir() else 'https://ultralytics.com/images/bus.jpg'
            LOGGER.warning(f"WARNING ⚠️ 'source' is missing. Using 'source={source}'.")
        overrides = self.overrides.copy() # overrides?
        overrides['conf'] = 0.25
        overrides.update(kwargs)  # prefer kwargs
        overrides['mode'] = kwargs.get('mode', 'predict')
        assert overrides['mode'] in ['track', 'predict'] # python에서 호출되면, 디폴트로 저장 안한다.
        overrides['save'] = kwargs.get('save', False)  # do not save by default if called in Python
        self.predictor = FastSAMPredictor(overrides=overrides)
        self.predictor.setup_model(model=self.model, verbose=False)
        try:
            return self.predictor(source, stream=stream)
        except Exception as e:
            return None

    def train(self, **kwargs): # 학습은 제공 안해?
        """Function trains models but raises an error as FastSAM models do not support training."""
        raise NotImplementedError("Currently, the training codes are on the way.")

    def val(self, **kwargs):
        """Run validation given dataset."""
        overrides = dict(task='segment', mode='val')
        overrides.update(kwargs)
        args = get_cfg(cfg=DEFAULT_CFG, overrides=overrides)
        args.imgsz = check_imgsz(args.imgsz, max_dim=1)
        validator = FastSAM(args=args)
        validator(model=self.model) # self.model 은 YOLO로 부터 온다.
        self.metrics = validator.metrics
        return validator.metrics

    @smart_inference_mode()
    def export(self, **kwargs):
        """
        Export model.

        Args:
            **kwargs : Any other args accepted by the predictors. 
            To see all args check 'configuration' section in docs
        """
        overrides = dict(task='detect')
        overrides.update(kwargs)
        args = get_cfg(cfg=DEFAULT_CFG, overrides=overrides)
        args.task = self.task
        if args.imgsz == DEFAULT_CFG.imgsz: # ultralytics.yolo.utils 디폴트 CFG의 imgsz라면??
            args.imgsz = self.model.args['imgsz']  # use trained imgsz unless custom value is passed
        if args.batch == DEFAULT_CFG.batch: # ultralytics.yolo.utils 디폴트 배치는 '1'인가 보다.
            args.batch = 1  # default to 1 if not modified
        return Exporter(overrides=args)(model=self.model) # Exporter??

    def info(self, detailed=False, verbose=True):
            """
            Logs model info.

            Args:
                detailed (bool): Show detailed information about model.
                verbose (bool): Controls verbosity.
            """
            return model_info(self.model, detailed=detailed, verbose=verbose, imgsz=640)

    def __call__(self, source=None, stream=False, **kwargs): # 클래스 그 자체를 호출할 때
        """Calls the 'predict' function with given arguments to perform object detection."""
        return self.predict(source, stream, **kwargs)

    def __getattr__(self, attr):
        """Raises error if object has no requested attribute."""
        name = self.__class__.__name__
        raise AttributeError(f"'{name}' object has no attribute '{attr}'. See valid attributes below.\n{self.__doc__}")